# Pretrain Policy with  `MPC dataset`

In [ ]:
import torch
import mujoco
import numpy as np
import matplotlib.pyplot as plt
from mujoco_parser import MuJoCoParserClass
from util import rpy2r
import pickle

from policy import GaussianPolicy

np.set_printoptions(precision=2,suppress=True,linewidth=100)
plt.rc('xtick',labelsize=6); plt.rc('ytick',labelsize=6)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
print ("MuJoCo version:[%s]"%(mujoco.__version__))


### Load  `Pretrained Model`

In [ ]:
checkpoint = torch.load("pretrained2.pth",map_location=torch.device('cpu'))
obs_dim = 99
action_dim = 26
hidden_dim = 512
policy = GaussianPolicy(
    input_dim=obs_dim,
    output_dim=action_dim,
    hidden_dim=hidden_dim,
    is_deterministic=False,)
policy.load_state_dict(checkpoint)

In [ ]:
policy

In [ ]:
with open(file='../data/dataset.pkl', mode='rb') as f:
    dataset = pickle.load(f)

action_batch = torch.Tensor(dataset['action'])
qpos_batch = torch.Tensor(dataset['qpos'])[:,2:]
qvel_batch = torch.Tensor(dataset['qvel'])
obs_batch = torch.cat((qpos_batch, qvel_batch), dim=1)

print("action : ", action_batch.shape)
print("qpos : ", qpos_batch.shape)
print("qvel : ", qvel_batch.shape)

obs_dim = obs_batch.shape[1]
action_dim = action_batch.shape[1]
hidden_dim = 512

del dataset

In [ ]:
diff = action_batch - policy(obs_batch)[0]

In [ ]:
idx = 0
mse=torch.nn.MSELoss()
# torch.sqrt(torch.mean(torch.square(diff),dim=1)).shape
torch.mean(torch.sqrt(torch.square(diff)),dim=1)

In [ ]:
xml_path = '../asset/smpl_rig/smpl_rig_rilab_mpc.xml'
env = MuJoCoParserClass(name='smplrig',rel_xml_path=xml_path,VERBOSE=True)
# Modify the color of body exluding 'world'
for body_name in env.body_names:
    if body_name in ['world']: continue
    body_idx = env.body_names.index(body_name)
    geom_idxs = [idx for idx,val in enumerate(env.model.geom_bodyid) if val==body_idx]
    for geom_idx in geom_idxs:
        env.model.geom(geom_idx).rgba = [0.3,0.3,0.5,0.5]
print ("Done.")

In [ ]:
# Set which joints to control
ctrl_joint_names = env.ctrl_names # <==
joint_idxs_fwd = env.get_idxs_fwd(joint_names=ctrl_joint_names)
joint_idxs_jac = env.get_idxs_jac(joint_names=ctrl_joint_names)
q_ctrl_init = env.get_qpos_joints(ctrl_joint_names)
n_ctrl_joint = len(ctrl_joint_names)

# import pandas as pd
# pkl_path = '../data/F01A0V1.pkl'
# pd.read_pickle(pkl_path)

import pickle
pkl_path = '../data/M02F4V1.pkl'
with open(pkl_path,'rb') as f:
    data = pickle.load(f)

print(data['length'])
print(data['root'].shape)
print(data['qpos'].shape)

In [ ]:
L = data['length']

# Initialize MuJoCo viewer
env.init_viewer(viewer_title='SMPL',viewer_width=1200,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=152,distance=3.0,elevation=-20,lookat=[0.02,-0.03,1.2])
env.reset()
tick = 0
q = data['qpos'][tick,:]
p_root = data['root'][tick,:]
# tick = min(tick+1,L-1)
# if tick==(L-1): tick = 0
# else: tick = tick + 1
env.set_p_root(root_name='base',p=p_root)
env.forward(q=q,INCREASE_TICK=True)
policy.eval()

while env.is_viewer_alive():
    qpos = env.data.qpos
    qvel = env.data.qvel
    state = np.concatenate((qpos, qvel))
    state = torch.Tensor(state)
    action, _ = policy(state)
    
    env.step(action.detach().numpy())
    if env.loop_every(tick_every=1):
        # Plot world frame
        env.plot_T(p=np.zeros(3),R=np.eye(3,3),
                   PLOT_AXIS=True,axis_len=0.5,axis_width=0.005)
        env.plot_T(p=np.array([0,0,0.5]),R=np.eye(3,3),
                   PLOT_AXIS=False,label="tick:[%d]"%(tick))
        # Plot foot
        # env.plot_geom_T(geom_name='rfoot',axis_len=0.3)
        # env.plot_geom_T(geom_name='lfoot',axis_len=0.3)
        # Plot revolute joints with arrow
        # env.plot_joint_axis(axis_len=0.1,axis_r=0.01)    
        env.render()
# Close MuJoCo viewer
env.close_viewer()
print ("Done.")